https://www.philschmid.de/fine-tune-llms-in-2024-with-trl

``` bash
pip install sentencepiece transformers trl datasets pandas numpy protobuf accelerate bitsandbytes flash-attn
```

In [1]:
import torch
torch.__version__
# должна быть '2.0.1+cu117'
# pip install torch==2.0.1

'2.0.1+cu117'

In [2]:
import torch
print(torch.cuda.get_device_properties(0))
print(torch.randn(1).cuda())
# _CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40384MB, multi_processor_count=108)
# tensor([-0.2955], device='cuda:0')

_CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40384MB, multi_processor_count=108)
tensor([-0.3421], device='cuda:0')


In [2]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset, load_dataset_builder
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, AutoConfig, pipeline, GenerationConfig, DataCollatorForLanguageModeling
from trl import SFTTrainer
import os
import numpy as np

pd.set_option('max_colwidth', 400)

In [4]:
from huggingface_hub import login
from dotenv import load_dotenv
load_dotenv()

login(
  token=os.getenv("HF_TOKEN"),
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Пример использования промпта

In [5]:
model_id="Open-Orca/Mistral-7B-OpenOrca" # pip install sentencepiece

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=False
)

chat = [
   {"role": "system", "content": "You're AI assistant"},
   {"role": "user", "content": "Hello, how are you?"},
   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
   {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

print(tokenizer.apply_chat_template(chat, tokenize=False))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<|im_start|>system
You're AI assistant<|im_end|>
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing great. How can I help you today?<|im_end|>
<|im_start|>user
I'd like to show off how chat templating works!<|im_end|>



# Подготовка датасета

## 0 variant

In [6]:
dataset = pd.read_csv('./dataset_5000.csv')

dataset = {'question': dataset['question'].to_list(), 'answer': dataset['answer'].to_list()}
dataset = Dataset.from_dict(dataset)
dataset = dataset.shuffle()

In [7]:
SYSTEM_PROMPT = 'Ты ассистент-помощник, который отвечает на вопросы человека. Ты должен быть вежлив и точен в своих ответах. Отвечай кратко и понятно.'

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

dataset = dataset.map(create_conversation, remove_columns=dataset.features, batched=False)

Map: 100%|██████████| 5000/5000 [00:00<00:00, 8472.92 examples/s]


In [8]:
dataset['messages'][0]

[{'content': 'Ты ассистент-помощник, который отвечает на вопросы человека. Ты должен быть вежлив и точен в своих ответах. Отвечай кратко и понятно.',
  'role': 'system'},
 {'content': 'Действительно ли все в этом мире имеет свою причину?',
  'role': 'user'},
 {'content': 'Никак не иначе. Причины чего именно Вы ставите под сомнение? Составьте список - и я Вам все причины назову. А если не "влом" - обясните: как представляете себе что-то без причины?',
  'role': 'assistant'}]

In [9]:
dataset = dataset.train_test_split(test_size=1/5)
print(dataset["train"][0]["messages"])

[{'content': 'Ты ассистент-помощник, который отвечает на вопросы человека. Ты должен быть вежлив и точен в своих ответах. Отвечай кратко и понятно.', 'role': 'system'}, {'content': 'Где лучше всего по вашему мнению хранить свои пароли?', 'role': 'user'}, {'content': 'Самый лучший способ - в голове.\nЧуть похуже - на бумажке, которая убрана в недоступное для посторонних место.\nЕсть различные менеджеры хранения паролей (например, KeePass или LastPass).\nМожно просто хранить в файле (в любом текстовом редакторе).\n.\nСамый надежный - это хранить в голове. Менее надёжный - использовать менеджеры.', 'role': 'assistant'}]


In [10]:
# сохранение датасета
dataset['train'].to_json("./dataset/train_dataset.json", orient="records")
dataset['test'].to_json("./dataset/test_dataset.json", orient="records")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 15.33ba/s]


4598874

In [11]:
from datasets import load_dataset

dataset = load_dataset(
    path="./dataset",
    data_files="train_dataset.json", # убираем если нужно загрузить train+test
    split="train" # убираем если нужно загрузить train+test
)
dataset

Generating train split: 4000 examples [00:00, 71022.79 examples/s]


Dataset({
    features: ['messages'],
    num_rows: 4000
})

## 1 variant

In [11]:
df = pd.read_csv('./dataset_5000.csv')
df.head(3)

,question,answer
0,Чем не занимается инженер-строитель?,"В каждой компании по разному! Но с уверенностью могу заявить, что инженер-строитель не управляет строительными машинами (а именно не сидит за рулем) и сам не производит строительные работы (не пачкает руки)\nКакой вопрос, такой ответ..."
1,Какие витамины содержат кремний?,"Кремний содержится в популярных витаминно -минеральных комплексах с кремнием, например, Доппельгерц актив, Витрум. Также кремний содержится в специально разработанных витаминах для восполнения дефицита кремния, например, Leader Vahva Piimaa, Piimax C + Biotini, Balance drink Si+, Piimax Pro-Vita."
2,"Почему в России певцы, актеры, шоумены все время «сбиваются» в стайки на концерты, огоньки, корпоративы. А в Европе и США — каждая звезда сама по себе?","Боюсь ошибиться, не сильна в теме телевидения и шоу-бизнеса, но предположу.\nМне кажется, многие форматы публичных - телевизионных и не только - выступлений артистов, в частности, музыкантов, пришли по большей части с запада, потому все, что так или иначе есть ""у нас"", изначально было или есть/остается также и ""у них"".\n1) Концерты.\n- всякие церемонии вручения музыкальных премий как вариант...."


In [44]:
SYSTEM_PROMPT = 'Ты ассистент-помощник, который отвечает на вопросы человека. Ты должен быть вежлив и точен в своих ответах. Отвечай кратко и понятно.'

df['messages'] = df[['question', 'answer']].apply(lambda x: np.array([{'role':'system', 'content': SYSTEM_PROMPT},
                                                                      {'role':'user', 'content': x['question']},
                                                                      {'role':'assistant', 'content': x['answer']}]), axis=1)


# НЕ НУЖНО ПРИМЕНЯТЬ ЧАТ ТЕМПЛЕЙТ!!!
# df['messages'] = df['chat_format'].apply(lambda x: tokenizer.apply_chat_template(x, tokenize=False))
# НЕ НУЖНО ПРИМЕНЯТЬ ЧАТ ТЕМПЛЕЙТ!!!


train_dataset = df[['messages']]
train_dataset.head(3)

,messages
0,"[{'role': 'system', 'content': 'Ты ассистент-помощник, который отвечает на вопросы человека. Ты должен быть вежлив и точен в своих ответах. Отвечай кратко и понятно.'}, {'role': 'user', 'content': 'Чем не занимается инженер-строитель?'}, {'role': 'assistant', 'content': 'В каждой компании по разному! Но с уверенностью могу заявить, что инженер-строитель не управляет строительными машинами (а и..."
1,"[{'role': 'system', 'content': 'Ты ассистент-помощник, который отвечает на вопросы человека. Ты должен быть вежлив и точен в своих ответах. Отвечай кратко и понятно.'}, {'role': 'user', 'content': 'Какие витамины содержат кремний?'}, {'role': 'assistant', 'content': 'Кремний содержится в популярных витаминно -минеральных комплексах с кремнием, например, Доппельгерц актив, Витрум. Также кремний..."
2,"[{'role': 'system', 'content': 'Ты ассистент-помощник, который отвечает на вопросы человека. Ты должен быть вежлив и точен в своих ответах. Отвечай кратко и понятно.'}, {'role': 'user', 'content': 'Почему в России певцы, актеры, шоумены все время «сбиваются» в стайки на концерты, огоньки, корпоративы. А в Европе и США — каждая звезда сама по себе?'}, {'role': 'assistant', 'content': 'Боюсь оши..."


In [48]:
data = Dataset.from_pandas(train_dataset)
data = data.shuffle()
data

Dataset({
    features: ['messages'],
    num_rows: 5000
})

In [49]:
data = data.train_test_split(test_size=1/5)
data

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 1000
    })
})

In [16]:
# сохранение датасета
data['train'].to_json("./dataset/train_dataset.json", orient="records")
data['test'].to_json("./dataset/test_dataset.json", orient="records")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 42.73ba/s]


4339678

In [17]:
from datasets import load_dataset

dataset = load_dataset(
    path="./dataset",
    data_files="train_dataset.json", # убираем если нужно загрузить train+test
    split="train" # убираем если нужно загрузить train+test
)
dataset

Generating train split: 4000 examples [00:00, 154025.39 examples/s]


Dataset({
    features: ['messages'],
    num_rows: 4000
})

# Finetuning

## Загрузка модели

In [7]:
model_id="./Mistral-7B-OpenOrca" # модель, которую будем дообучать
lora_model="./OpenOrca-7b-aesedeu-lora" # директория с LoRA-адаптерами, которые получим на выходе

In [13]:
def get_model_and_tokenizer(model_id):

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.padding_side = 'right' # to prevent warnings
    tokenizer.pad_token = tokenizer.eos_token # Устанавливает токен для дополнения (pad_token) равным токену конца строки (eos_token). Это полезно для моделей, которые используют один и тот же токен для обозначения конца строки и дополнения.

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=False, # Загружает модель в 4-битном формате для уменьшения использования памяти.
        load_in_8bit=True,
        bnb_4bit_quant_type="fp4", # Указывает тип квантования, в данном случае "nf4" (nf4/dfq/qat/ptq/fp4)
        bnb_4bit_compute_dtype="float16", # Устанавливает тип данных для вычислений в 4-битном формате как float16.
        bnb_4bit_use_double_quant=False # Указывает, что не используется двойное квантование.
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
        device_map="auto",
        attn_implementation="flash_attention_2"
    )

    model.config.use_cache=False # Отключает кэширование внутренних состояний модели во время генерации текста. Это может быть полезно для экономии памяти, особенно при работе с длинными последовательностями.
    model.config.pretraining_tp=1 # параметр, связанный с техниками распределенного обучения 
    
    return model, tokenizer

model, tokenizer = get_model_and_tokenizer(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.91s/it]


## Настройка параметров обучения

In [14]:
# НАСТРОИТЬ!!!

peft_config = LoraConfig(
        r=8, # определяет ранг матриц адаптации в LoRA. Этот параметр контролирует количество параметров адаптации, которые добавляются к модели. Более низкий ранг означает меньше дополнительных параметров и, следовательно, меньшее влияние на исходные веса модели.
        lora_alpha=32, # масштабирующий коэффициент для матриц адаптации. Этот коэффициент управляет степенью, с которой адаптированные веса влияют на поведение модели.
        # lora_dropout=0.05,
        bias="none", # указывает, должен ли добавляться смещение (bias) к параметрам LoRA
        task_type="CAUSAL_LM", # "CAUSAL_LM" означает причинно-следственное (каузальное) языковое моделирование ????????????
        # target_modules="all-linear"
    )

training_arguments = TrainingArguments(
        output_dir=lora_model, # Путь к каталогу, где будут сохраняться обученная модель и другие выходные данные.
        per_device_train_batch_size=2, # Размер пакета (batch size) для обучения на каждом устройстве. Определяет количество образцов данных, обрабатываемых за один шаг обучения на каждом устройстве.
        gradient_accumulation_steps=3, # Количество шагов накопления градиента. Это позволяет эффективно увеличить размер пакета, не увеличивая использование памяти.
        gradient_checkpointing=True, # чекпоинты градиентов для сохранения памяти
        optim="adamw_torch", # Оптимизатор, используемый для обучения. В данном случае используется "paged_adamw_32bit", что представляет собой определённую версию оптимизатора AdamW с 32-битной точностью.
        learning_rate=2e-4, # Скорость обучения. Определяет, насколько сильно веса модели обновляются во время обучения.
        lr_scheduler_type="cosine", # "constant" / Тип планировщика скорости обучения. "cosine" означает использование косинусного расписания с понижением скорости обучения.
        bf16=True, # use bfloat16 precision
        tf32=True, # use tf32 precision
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        save_strategy="steps", # Стратегия сохранения модели. "epoch" означает, что модель будет сохраняться после каждой эпохи обучения.
        logging_steps=10, # Количество шагов обучения между логированием метрик обучения.
        num_train_epochs=3, # Количество эпох обучения, то есть сколько раз обучающий набор данных будет проходить через модель.
        # eval_steps=50,
        save_steps=100,
        max_steps=500, # Максимальное количество шагов обучения. Обучение закончится, когда будет достигнуто это число шагов, даже если не все эпохи были завершены.
        # report_to="tensorboard",
        # push_to_hub=True
    )

trainer = SFTTrainer(
        model=model,
        peft_config=peft_config,
        args=training_arguments,
        train_dataset=dataset,
        # eval_dataset=dataset,
        # dataset_text_field="messages", # не передаем это поле, если датасет уже был предобработан в нужный формат
        dataset_kwargs={
            "add_special_tokens": False,  # We template with special tokens
            "append_concat_token": False, # No need to add additional separator token
        },
        # packing=True,
        tokenizer=tokenizer,
        max_seq_length=2048
    )

Map: 100%|██████████| 4000/4000 [00:01<00:00, 2937.57 examples/s]


In [15]:
# trainer.train(resume_from_checkpoint=True)
trainer.train()

# save model
trainer.save_model()

/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs

Step,Training Loss
10,2.060600
20,1.530600
30,1.454400
40,1.418700
50,1.495000
60,1.390900
70,1.345200
80,1.416100
90,1.356900
100,1.403100


/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./Mistral-7B-OpenOrca - will assume that the vocabulary was not modified.
  warnings.warn(
/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:14

In [45]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

NameError: name 'model' is not defined

# Объединение модели с LoRA-адаптером

In [8]:
model_id="./Mistral-7B-OpenOrca" # модель, которую будем дообучать
lora_model="./OpenOrca-7b-aesedeu-lora" # директория с LoRA-адаптерами, которые получим на выходе
lora_adapters = lora_model + "/checkpoint-500"

In [37]:
merged_model_path = lora_model + "-merged"

### COMMENT IN TO MERGE PEFT AND BASE MODEL ####
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM

# Load PEFT model on CPU
config = PeftConfig.from_pretrained(lora_adapters)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(lora_adapters)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, lora_adapters)

model = AutoPeftModelForCausalLM.from_pretrained(
    lora_adapters,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(merged_model_path, safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained(merged_model_path)

('./OpenOrca-7b-aesedeu-lora-merged/tokenizer_config.json',
 './OpenOrca-7b-aesedeu-lora-merged/special_tokens_map.json',
 './OpenOrca-7b-aesedeu-lora-merged/tokenizer.model',
 './OpenOrca-7b-aesedeu-lora-merged/added_tokens.json',
 './OpenOrca-7b-aesedeu-lora-merged/tokenizer.json')

# Инференс обученной модели

## модель + адаптеры

In [4]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

model_id = "./Mistral-7B-OpenOrca"

model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="cuda",
  # load_in_8bit=True,
  torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(
  model,
  "./OpenOrca-7b-aesedeu-lora/checkpoint-500/",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.padding_side = 'right' # to prevent warnings

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

## только адаптеры

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# peft_model_id = "./OpenOrca-7b-aesedeu-lora-merged"
peft_model_id = "OpenOrca-7b-aesedeu-lora/checkpoint-500"
model_id="./Mistral-7B-OpenOrca" 

model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="cuda",
  # load_in_8bit=True,
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.28s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been a

## объединенная модель

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

model_id = "./OpenOrca-7b-aesedeu-lora-merged"

model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="cuda",
  # load_in_8bit=True,
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.padding_side = 'right' # to prevent warnings

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [13]:
from datasets import load_dataset
from random import randint

eval_dataset = load_dataset(
    path="./dataset",
    data_files="test_dataset.json", # убираем если нужно загрузить train+test
    split="train" # убираем если нужно загрузить train+test
)

rand_idx = randint(0, len(eval_dataset))

In [15]:
# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)

outputs = pipe(
    prompt,
    repetition_penalty=1.1,
    max_new_tokens=256,
    do_sample=False,
    temperature=0.1,
    top_k=10,
    top_p=0.9,
    eos_token_id=pipe.tokenizer.eos_token_id,
    pad_token_id=pipe.tokenizer.pad_token_id,
    # no_repeat_ngram_size=15,
    # bos_token_id=1,
    # eos_token_id=2
)

/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top

In [16]:
print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print("==================================")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print("==================================")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
Торжество православия значит - торговля в храмах РПЦ?
Original Answer:
По вопросу сразу видно хейтера и хайпожора))) Хайповать, конечно не мешки ворочать. Если отвечать по существу, то Торжество Православия - праздник, который был установлен после решений последнего Вселенского Собора об иконопочитании в качестве аксиомы правосланого вероучения, а иконоборчество было осуждено как ересь (неправославная доктрина). Но ведь автора не это интересует, правда?
Generated Answer:
Не совсем так... Всё дело не только историческими фактами (например: продажа мощевиков святынь), но также с текущией ситуационой рынке реализации духовных услуг как таковых! Поэтому если вы хотите осудиться церкви за это я ваш человек) Но все же стоит обращатся к истории для того чтобы разобрался что таки "торги" были при них или без их?! Ибо они далекий разумеется больше чем обязанности перед Богом!!! А себе доверь!! :) Есть много способы помочью своим согражданиным!!!! Спасительный путей избавок можемы найдено

https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/fine-tune-llms-in-2024-with-trl.ipynb